# Scratch ML : Random Forest Algorithm
**Training**
- Extract a subset of whole dataset
- Create a decision Tree
- Repeat as many times are number of trees

**Testing**
- Get Predictions from each tree
- Classfication : Majority vaote is considered
- Get the mean of the predictions

In [2]:
# Importing math class
import numpy as np
from collections import Counter

In [3]:
# Class Node
class Node:
    # setting Up constrcutor
    def __init__(self, feature=None, threshold=None, left=None, right=None,*, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

In [4]:
# Class Decistion Tree
class DecisionTree:

    # Setting up constructor
    def __init__(self, min_sample_split=2, max_depth=5, n_features=None):
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        # Extract Root of tree
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X,y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))
        
        # Checking Stopping Criteria
        if (depth >= self.max_depth) or (n_labels == 1) or (n_samples < self.min_sample_split):
            leaf_value = self._most_common_label(y)
            return Node(value = leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)
        

        # Find best split 
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # Create Child Nodes
        left_idxs , right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth=+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth=+1)

        # Returning Node
        return Node(best_feature, best_thresh, left, right)

    
    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None

        # Travering all opts
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thresh in thresholds:
                # Calulcate IG
                gain = self._information_gain(y, X_column, thresh)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = thresh

        return split_idx, split_thresh
            
    def predict(self, X_test):
        return np.array([self._traverse_tree(x, self.root) for x in X_test])


    # HELPER FUNCTIONS
    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_ent = self._entropy(y)
        
        # create children
        left_idxs , right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0  or len(left_idxs) == 0 :
            return 0

        # calculate weighter avg entropy of children
        n = len(y)
        n_l , n_r = len(left_idxs), len(right_idxs)
        e_l , e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_ent = (n_l/n) * e_l + (n_r/n) * e_r
        
        # calculate the IG
        information_gain = parent_ent - child_ent
        return information_gain


    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    
    def _entropy(self, y):
        hist = np.bincount(y)
        proba = hist / len(y)
        return -np.sum([p*np.log(p) for p in proba if p>0])
            
        
    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    
    def _traverse_tree(self, X, node):
        if node.is_leaf_node():
            return node.value

        if X[node.feature] <= node.threshold:
            return self._traverse_tree(X, node.left)
        return self._traverse_tree(X, node.right)

## Setting up Random Forest

In [29]:
class RandomForest:

    # Setting up constructor
    def __init__(self, n_trees = 10, max_depth =2, min_samples_split=2, n_features=None ):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth, 
                         min_sample_split=self.min_samples_split,
                        n_features=self.n_features)

            # Collecting Samples
            X_sample, y_sample = self._bootstrap_samples(X,y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)
            

    def _bootstrap_samples(self,X,y):
        # Selecting Samples randomly
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value
    
    def predict(self, X_test):
        preds = np.array([tree.predict(X_test) for tree in self.trees])
        tree_preds = np.swapaxes(preds, 0,1)
        preds = np.array([self._most_common_label(pred) for pred in tree_preds]) 
        return preds

## Setting Up Accuracy Model

In [10]:
def accuracy (preds, y_test):
    return np.sum(preds == y_test) / len(y_test)

## Setting up Dataset : Breast Cancer

In [11]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [12]:
data = datasets.load_breast_cancer()
X,y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

## Setting up Deicsion Tree Model

In [32]:
RF = RandomForest()
RF.fit(X_train, y_train)
preds = RF.predict(X_test) 
print(f"Accuracy Score : {accuracy(preds, y_test)}")

Accuracy Score : 0.9361702127659575


In [31]:
RF = RandomForest(n_trees=100, max_depth=5)
RF.fit(X_train, y_train)
preds = RF.predict(X_test)
print(f"Accuracy Score : {accuracy(preds, y_test)}")

Accuracy Score : 0.9361702127659575
